In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier,StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, plot_confusion_matrix,cohen_kappa_score,log_loss
from sklearn.model_selection import GridSearchCV

from imblearn.over_sampling import SMOTE

%matplotlib inline

from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier
import mlxtend
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

from sklearn.feature_selection import RFE,f_classif,SelectKBest,chi2
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import LabelBinarizer,OrdinalEncoder
from xverse.transformer import WOE
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import ttest_ind

from matplotlib.colors import ListedColormap

pd.options.display.max_columns = None
pd.options.display.max_rows = None
from scipy import stats

In [44]:
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols
from scipy.stats import ttest_ind

In [243]:
from xgboost import XGBClassifier

In [249]:
from sklearn.ensemble import StackingClassifier

In [268]:
import sklearn

In [2]:
train = pd.read_csv('Data_Train.csv')
train.head()

,ID,Asst_Reg,GGGrade,Experience,Validation,Yearly Income,Home Status,Unpaid 2 years,Already Defaulted,Designation,Debt to Income,Postal Code,Lend Amount,Deprecatory Records,Interest Charged,Usage Rate,Inquiries,Present Balance,Gross Collection,Sub GGGrade,File Status,State,Account Open,Total Unpaid CL,Duration,Unpaid Amount,Reason,Claim Type,Due Fee,Loan/No Loan
0,95074,421802,II,>10yrs,Vfied,633600.00,MOR??AGE,0,0,GLENDALE NISSAN,12.5496,91300.0,42023.25,0,15.39,88.924,3,607161.90,0.0,19,fully paid,California,17,58598.0,3 years,31216.05,debt consolidation,I,0.0,0
1,134131,3964312,IV,7yrs,Source Verified,85483.20,REN?,0,0,Business Teacher,9.0048,27400.0,38133.00,0,9.94,102.856,0,269234.06,0.0,30,whole,NC,15,18924.0,5 years,11660.49,debt consolidation,I,0.0,0
2,92602,4247560,III,<1yr,Source Verified,79200.00,REN?,0,0,driver,24.9144,32800.0,17100.00,0,22.35,60.372,0,22476.53,0.0,25,whole,Florida,7,15604.0,5 years,5637.87,major purchase,I,0.0,0
3,22591,197179,III,<1yr,Vfied,61600.00,REN?,0,0,AvePoint,16.5816,7000.0,5130.00,0,10.36,116.272,1,15242.09,0.0,22,fully paid,NewJersey,9,22410.0,3 years,15607.17,major purchase,I,0.0,1
4,125060,4646684,V,2yrs,Source Verified,68053.92,REN?,0,0,Lead Tester,40.0512,70800.0,19665.00,0,13.68,127.280,1,65433.94,0.0,32,fully paid,LA,10,36022.0,5 years,27472.86,debt consolidation,I,0.0,0


In [3]:
train.tail()

,ID,Asst_Reg,GGGrade,Experience,Validation,Yearly Income,Home Status,Unpaid 2 years,Already Defaulted,Designation,Debt to Income,Postal Code,Lend Amount,Deprecatory Records,Interest Charged,Usage Rate,Inquiries,Present Balance,Gross Collection,Sub GGGrade,File Status,State,Account Open,Total Unpaid CL,Duration,Unpaid Amount,Reason,Claim Type,Due Fee,Loan/No Loan
87495,120884,4535984,II,>10yrs,Not Vfied,75680.00,REN?,0,0,Engineer Support II,19.3200,48000.0,10260.0,0,8.35,51.084,1,15100.14,0.0,17,whole,MI,12,32702.0,3 years,10012.05,debt consolidation,I,0.0,0
87496,33259,2624710,IV,3yrs,Source Verified,90423.52,REN?,0,0,Dental Hygienist,48.8208,1400.0,27360.0,0,24.18,129.172,0,102811.88,0.0,28,whole,MA,23,32204.0,5 years,24921.54,debt consolidation,I,0.0,0
87497,102153,5966919,II,>10yrs,Source Verified,91520.00,MOR??AGE,0,0,Interface Coordinator,7.7952,6400.0,8550.0,3,11.03,83.420,1,14584.11,0.0,16,fully paid,CT,8,29050.0,3 years,14517.90,debt consolidation,I,0.0,0
87498,115343,2630667,III,1yrs,Source Verified,61600.00,MOR??AGE,0,0,PARTS CLERK,8.2992,16800.0,23769.0,1,22.52,56.760,1,59281.66,0.0,25,whole,PA,11,24900.0,3 years,8454.24,debt consolidation,I,0.0,0
87499,104968,6637289,III,2yrs,Vfied,NaN,REN?,0,0,Outside sales rep,15.1872,92800.0,28471.5,1,15.53,67.080,0,10445.85,0.0,25,whole,California,7,21746.0,5 years,8717.58,debt consolidation,I,0.0,0


In [4]:
test = pd.read_csv('Data_Test.csv')
test.head()

,ID,Asst_Reg,GGGrade,Experience,Validation,Yearly Income,Home Status,Unpaid 2 years,Already Defaulted,Designation,Debt to Income,Postal Code,Lend Amount,Deprecatory Records,Interest Charged,Usage Rate,Inquiries,Present Balance,Gross Collection,Sub GGGrade,File Status,State,Account Open,Total Unpaid CL,Duration,Unpaid Amount,Reason,Claim Type,Due Fee
0,198980,1773284,II,2yrs,Source Verified,140800.00,OWN,0,0,processor,16.9176,90600.0,17955.00,0,8.88,78.432,1,59769.30,0.0,17,fully paid,California,23,33532.0,3 years,15755.94,major purchase,I,0.0
1,167688,5298273,II,>10yrs,Vfied,146080.00,MOR??AGE,0,0,Speech Language Pathologist,28.6944,32300.0,59850.00,0,10.10,76.368,1,532962.13,0.0,19,whole,Florida,17,149234.0,5 years,68237.55,debt consolidation,I,0.0
2,198768,189781,VI,>10yrs,Vfied,202222.24,MOR??AGE,0,0,Raytheon,51.1560,88000.0,59850.00,0,19.30,157.552,2,288816.48,0.0,37,whole,NM,15,70218.0,5 years,66272.76,debt consolidation,I,0.0
3,192564,1530906,II,>10yrs,Vfied,105600.00,MOR??AGE,1,0,Computer/Network Technician,27.0144,19900.0,19323.00,0,8.82,32.852,2,424664.30,0.0,19,fully paid,DE,18,79182.0,3 years,15595.20,credit card,I,0.0
4,151199,4410316,III,4yrs,Not Vfied,151360.00,MOR??AGE,0,0,NDE Tech,10.3824,34900.0,25350.75,1,13.52,108.532,1,251704.07,0.0,25,fully paid,Florida,7,13612.0,3 years,8850.96,debt consolidation,I,0.0


In [5]:
test.tail()

,ID,Asst_Reg,GGGrade,Experience,Validation,Yearly Income,Home Status,Unpaid 2 years,Already Defaulted,Designation,Debt to Income,Postal Code,Lend Amount,Deprecatory Records,Interest Charged,Usage Rate,Inquiries,Present Balance,Gross Collection,Sub GGGrade,File Status,State,Account Open,Total Unpaid CL,Duration,Unpaid Amount,Reason,Claim Type,Due Fee
37495,191405,5774816,IV,4yrs,Vfied,308000.0,MOR??AGE,1,0,Regional Manager,29.4336,91700.0,59850.00,0,19.15,52.116,4,848107.83,0.0,29,fully paid,California,19,132468.0,3 years,57683.43,debt consolidation,I,0.0
37496,164868,1423834,IV,8yrs,Source Verified,158400.0,MOR??AGE,0,0,Clinical Coordinator,31.6008,19500.0,28215.00,0,19.02,165.120,1,614710.30,0.0,28,fully paid,PA,14,51958.0,5 years,51395.76,credit card,I,0.0
37497,163156,1895531,V,>10yrs,Not Vfied,52800.0,OWN,0,0,administrative,46.9728,90800.0,4916.25,1,18.68,54.868,3,47287.72,0.0,33,fully paid,California,9,8466.0,3 years,2783.88,debt consolidation,I,0.0
37498,169610,2168910,VI,9yrs,Not Vfied,52800.0,OWN,5,0,ParaEducator,48.0480,37000.0,15390.00,1,10.47,137.772,1,107614.80,0.0,40,whole,TN,9,17264.0,3 years,NaN,home improvement,I,0.0
37499,175557,4140268,II,>10yrs,Source Verified,121440.0,MOR??AGE,0,0,School Teacher,27.2664,91900.0,24624.00,0,9.80,104.060,0,783477.16,0.0,19,fully paid,California,15,47310.0,3 years,29504.34,debt consolidation,I,0.0


In [6]:
train.shape

(87500, 30)

In [7]:
test.shape

(37500, 29)

In [8]:
sample = pd.read_csv('Sample_Submission.csv')
sample.head()

,ID,Loan/No Loan
0,198980,0
1,167688,0
2,198768,0
3,192564,0
4,151199,0


In [9]:
sample.loc[sample['Loan/No Loan']==1,'ID']

Series([], Name: ID, dtype: int64)

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87500 entries, 0 to 87499
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   87500 non-null  int64  
 1   Asst_Reg             87500 non-null  int64  
 2   GGGrade              87500 non-null  object 
 3   Experience           87500 non-null  object 
 4   Validation           87500 non-null  object 
 5   Yearly Income        81925 non-null  float64
 6   Home Status          87500 non-null  object 
 7   Unpaid 2 years       87500 non-null  int64  
 8   Already Defaulted    87500 non-null  int64  
 9   Designation          86086 non-null  object 
 10  Debt to Income       84011 non-null  float64
 11  Postal Code          86111 non-null  float64
 12  Lend Amount          87500 non-null  float64
 13  Deprecatory Records  87500 non-null  int64  
 14  Interest Charged     87500 non-null  float64
 15  Usage Rate           87500 non-null 

In [11]:
train.describe()

,ID,Asst_Reg,Yearly Income,Unpaid 2 years,Already Defaulted,Debt to Income,Postal Code,Lend Amount,Deprecatory Records,Interest Charged,Usage Rate,Inquiries,Present Balance,Gross Collection,Sub GGGrade,Account Open,Total Unpaid CL,Unpaid Amount,Due Fee,Loan/No Loan
count,87500.000000,8.750000e+04,8.192500e+04,87500.000000,87500.000000,84011.000000,86111.000000,87500.000000,87500.000000,87500.000000,87500.000000,87500.000000,8.750000e+04,87500.000000,87500.000000,87500.00000,8.331400e+04,8.264800e+04,87500.000000,87500.000000
mean,83946.253509,3.798914e+06,1.348596e+05,0.332937,0.005726,30.942660,51155.385491,25920.535680,0.202400,16.518375,95.517348,0.670640,2.363895e+05,39.461697,22.024891,12.77944,5.326864e+04,2.955649e+04,0.316407,0.188709
std,36100.102950,2.289038e+06,9.882473e+04,0.876080,0.083505,14.079813,31211.229930,14433.837213,0.646414,4.902011,40.554035,0.959572,2.550568e+05,391.919578,6.452697,5.37116,5.325551e+04,3.745625e+04,3.546087,0.391279
min,21560.000000,2.484700e+04,8.800000e+03,0.000000,0.000000,0.000000,1000.000000,1710.000000,0.000000,8.000000,0.000000,0.000000,0.000000e+00,0.000000,11.000000,2.00000,3.320000e+02,0.000000e+00,0.000000,0.000000
25%,52616.750000,1.483874e+06,8.324659e+04,0.000000,0.000000,20.428800,22900.000000,15048.000000,0.000000,12.310000,66.048000,0.000000,5.173493e+04,0.000000,17.000000,9.00000,2.324000e+04,1.144674e+04,0.000000,0.000000
50%,83900.000000,4.132010e+06,1.144000e+05,0.000000,0.000000,30.139200,47700.000000,23940.000000,0.000000,16.490000,97.008000,0.000000,1.388062e+05,0.000000,21.000000,12.00000,3.967400e+04,2.089278e+04,0.000000,0.000000
75%,115261.250000,5.952909e+06,1.601600e+05,0.000000,0.000000,40.908000,80200.000000,34200.000000,0.000000,20.770000,126.592000,1.000000,3.534755e+05,0.000000,26.000000,16.00000,6.640000e+04,3.644908e+04,0.000000,0.000000
max,146559.000000,7.351847e+06,8.264031e+06,18.000000,6.000000,639.290400,99900.000000,59850.000000,86.000000,25.000000,1534.756000,8.000000,5.629234e+06,26308.470000,45.000000,83.00000,1.710630e+06,1.338778e+06,169.050000,1.000000


In [12]:
train.describe(include = 'object')

,GGGrade,Experience,Validation,Home Status,Designation,File Status,State,Duration,Reason,Claim Type
count,87500,87500,87500,87500,86086,87500,87500,87500,87500,87500
unique,7,11,3,5,40354,2,49,2,13,2
top,II,>10yrs,Source Verified,MOR??AGE,School Teacher,whole,California,3 years,debt consolidation,I
freq,24966,30849,34504,44160,1585,46300,12625,60061,52559,87454


In [13]:
(train.isnull().sum()/train.shape[0])*100

ID                     0.000000
Asst_Reg               0.000000
GGGrade                0.000000
Experience             0.000000
Validation             0.000000
Yearly Income          6.371429
Home Status            0.000000
Unpaid 2 years         0.000000
Already Defaulted      0.000000
Designation            1.616000
Debt to Income         3.987429
Postal Code            1.587429
Lend Amount            0.000000
Deprecatory Records    0.000000
Interest Charged       0.000000
Usage Rate             0.000000
Inquiries              0.000000
Present Balance        0.000000
Gross Collection       0.000000
Sub GGGrade            0.000000
File Status            0.000000
State                  0.000000
Account Open           0.000000
Total Unpaid CL        4.784000
Duration               0.000000
Unpaid Amount          5.545143
Reason                 0.000000
Claim Type             0.000000
Due Fee                0.000000
Loan/No Loan           0.000000
dtype: float64

In [14]:
(test.isnull().sum()/test.shape[0])*100

ID                     0.000000
Asst_Reg               0.000000
GGGrade                0.000000
Experience             0.000000
Validation             0.000000
Yearly Income          6.466667
Home Status            0.000000
Unpaid 2 years         0.000000
Already Defaulted      0.000000
Designation            1.562667
Debt to Income         4.029333
Postal Code            1.629333
Lend Amount            0.000000
Deprecatory Records    0.000000
Interest Charged       0.000000
Usage Rate             0.000000
Inquiries              0.000000
Present Balance        0.000000
Gross Collection       0.000000
Sub GGGrade            0.000000
File Status            0.000000
State                  0.000000
Account Open           0.000000
Total Unpaid CL        4.837333
Duration               0.000000
Unpaid Amount          5.517333
Reason                 0.000000
Claim Type             0.000000
Due Fee                0.000000
dtype: float64

#### All the null values are upto max 6.5% range, in both train and test data, thus can be imputed by median or mode

### Dealing with Missing values

In [16]:
## From the above describe values, we may infer that Yearly Income might be slightly right skewed.
## Thus we can impute median values to be on the safer side.

train.loc[train['Yearly Income'].isna(),'Yearly Income'] = train['Yearly Income'].median()

In [17]:
train.Designation.nunique()    ## We can drop this variable cuz of redundancy.

40354

In [18]:
## From the above describe values, we may infer that Debt to Income ratio might be slightly right skewed.
## Thus we can impute median values to be on the safer side.

train.loc[train['Debt to Income'].isna(),'Debt to Income'] = train['Debt to Income'].median()

In [19]:
train['Postal Code'].nunique()   ## We should impute with mode; Regional / Demographic variable

865

In [20]:
train.loc[train['Postal Code'].isna(),'Postal Code'] = train['Postal Code'].mode()

In [21]:
## We may impute median values for Total unpaid CL

train.loc[train['Total Unpaid CL'].isna(),'Total Unpaid CL'] = train['Total Unpaid CL'].median()

In [22]:
## Doing similar median imputation with the last default variable


train.loc[train['Unpaid Amount'].isna(),'Unpaid Amount'] = train['Unpaid Amount'].median()

In [23]:
(train.isnull().sum()/train.shape[0])*100

ID                     0.000000
Asst_Reg               0.000000
GGGrade                0.000000
Experience             0.000000
Validation             0.000000
Yearly Income          0.000000
Home Status            0.000000
Unpaid 2 years         0.000000
Already Defaulted      0.000000
Designation            1.616000
Debt to Income         0.000000
Postal Code            1.587429
Lend Amount            0.000000
Deprecatory Records    0.000000
Interest Charged       0.000000
Usage Rate             0.000000
Inquiries              0.000000
Present Balance        0.000000
Gross Collection       0.000000
Sub GGGrade            0.000000
File Status            0.000000
State                  0.000000
Account Open           0.000000
Total Unpaid CL        0.000000
Duration               0.000000
Unpaid Amount          0.000000
Reason                 0.000000
Claim Type             0.000000
Due Fee                0.000000
Loan/No Loan           0.000000
dtype: float64

In [29]:
## There may be some dilema with Postal Code, need to recheck

# We should ideally change the dtype of above to str/object

train['Postal Code'] = train['Postal Code'].astype(str)

In [32]:
## Applying mode values now

train.loc[train['Postal Code'].isna(),'Postal Code'] = train['Postal Code'].mode()

In [34]:
(train.isnull().sum()/train.shape[0])*100

ID                     0.000
Asst_Reg               0.000
GGGrade                0.000
Experience             0.000
Validation             0.000
Yearly Income          0.000
Home Status            0.000
Unpaid 2 years         0.000
Already Defaulted      0.000
Designation            1.616
Debt to Income         0.000
Postal Code            0.000
Lend Amount            0.000
Deprecatory Records    0.000
Interest Charged       0.000
Usage Rate             0.000
Inquiries              0.000
Present Balance        0.000
Gross Collection       0.000
Sub GGGrade            0.000
File Status            0.000
State                  0.000
Account Open           0.000
Total Unpaid CL        0.000
Duration               0.000
Unpaid Amount          0.000
Reason                 0.000
Claim Type             0.000
Due Fee                0.000
Loan/No Loan           0.000
dtype: float64

In [36]:
## We may initially remove 'Designation' variable.

train.std()

ID                     3.610010e+04
Asst_Reg               2.289038e+06
Yearly Income          9.575509e+04
Unpaid 2 years         8.760799e-01
Already Defaulted      8.350455e-02
Debt to Income         1.379714e+01
Lend Amount            1.443384e+04
Deprecatory Records    6.464135e-01
Interest Charged       4.902011e+00
Usage Rate             4.055404e+01
Inquiries              9.595719e-01
Present Balance        2.550568e+05
Gross Collection       3.919196e+02
Sub GGGrade            6.452697e+00
Account Open           5.371160e+00
Total Unpaid CL        5.204695e+04
Unpaid Amount          3.645688e+04
Due Fee                3.546087e+00
Loan/No Loan           3.912792e-01
dtype: float64

In [37]:
train1 = train.drop('Designation', axis = 1)

In [38]:
## Applying similar treatment on test data

In [39]:
test.loc[test['Yearly Income'].isna(),'Yearly Income'] = test['Yearly Income'].median()
test.loc[test['Debt to Income'].isna(),'Debt to Income'] = test['Debt to Income'].median()
test.loc[test['Total Unpaid CL'].isna(),'Total Unpaid CL'] = test['Total Unpaid CL'].median()
test.loc[test['Unpaid Amount'].isna(),'Unpaid Amount'] = test['Unpaid Amount'].median()

test['Postal Code'] = test['Postal Code'].astype(str)

test.loc[train['Postal Code'].isna(),'Postal Code'] = test['Postal Code'].mode()

test1 = test.drop('Designation', axis = 1)

In [40]:
(train1.isnull().sum()/train1.shape[0])*100

ID                     0.0
Asst_Reg               0.0
GGGrade                0.0
Experience             0.0
Validation             0.0
Yearly Income          0.0
Home Status            0.0
Unpaid 2 years         0.0
Already Defaulted      0.0
Debt to Income         0.0
Postal Code            0.0
Lend Amount            0.0
Deprecatory Records    0.0
Interest Charged       0.0
Usage Rate             0.0
Inquiries              0.0
Present Balance        0.0
Gross Collection       0.0
Sub GGGrade            0.0
File Status            0.0
State                  0.0
Account Open           0.0
Total Unpaid CL        0.0
Duration               0.0
Unpaid Amount          0.0
Reason                 0.0
Claim Type             0.0
Due Fee                0.0
Loan/No Loan           0.0
dtype: float64

In [41]:
(test1.isnull().sum()/test1.shape[0])*100

ID                     0.0
Asst_Reg               0.0
GGGrade                0.0
Experience             0.0
Validation             0.0
Yearly Income          0.0
Home Status            0.0
Unpaid 2 years         0.0
Already Defaulted      0.0
Debt to Income         0.0
Postal Code            0.0
Lend Amount            0.0
Deprecatory Records    0.0
Interest Charged       0.0
Usage Rate             0.0
Inquiries              0.0
Present Balance        0.0
Gross Collection       0.0
Sub GGGrade            0.0
File Status            0.0
State                  0.0
Account Open           0.0
Total Unpaid CL        0.0
Duration               0.0
Unpaid Amount          0.0
Reason                 0.0
Claim Type             0.0
Due Fee                0.0
dtype: float64

#### Missing value problem dealt with

### Encoding,Preprocessing and Understanding each Variables (Data Cleaning and EDA)

In [42]:
train1.head()

,ID,Asst_Reg,GGGrade,Experience,Validation,Yearly Income,Home Status,Unpaid 2 years,Already Defaulted,Debt to Income,Postal Code,Lend Amount,Deprecatory Records,Interest Charged,Usage Rate,Inquiries,Present Balance,Gross Collection,Sub GGGrade,File Status,State,Account Open,Total Unpaid CL,Duration,Unpaid Amount,Reason,Claim Type,Due Fee,Loan/No Loan
0,95074,421802,II,>10yrs,Vfied,633600.00,MOR??AGE,0,0,12.5496,91300.0,42023.25,0,15.39,88.924,3,607161.90,0.0,19,fully paid,California,17,58598.0,3 years,31216.05,debt consolidation,I,0.0,0
1,134131,3964312,IV,7yrs,Source Verified,85483.20,REN?,0,0,9.0048,27400.0,38133.00,0,9.94,102.856,0,269234.06,0.0,30,whole,NC,15,18924.0,5 years,11660.49,debt consolidation,I,0.0,0
2,92602,4247560,III,<1yr,Source Verified,79200.00,REN?,0,0,24.9144,32800.0,17100.00,0,22.35,60.372,0,22476.53,0.0,25,whole,Florida,7,15604.0,5 years,5637.87,major purchase,I,0.0,0
3,22591,197179,III,<1yr,Vfied,61600.00,REN?,0,0,16.5816,7000.0,5130.00,0,10.36,116.272,1,15242.09,0.0,22,fully paid,NewJersey,9,22410.0,3 years,15607.17,major purchase,I,0.0,1
4,125060,4646684,V,2yrs,Source Verified,68053.92,REN?,0,0,40.0512,70800.0,19665.00,0,13.68,127.280,1,65433.94,0.0,32,fully paid,LA,10,36022.0,5 years,27472.86,debt consolidation,I,0.0,0


In [43]:
train1.GGGrade.unique()

array(['II', 'IV', 'III', 'V', 'VII', 'VI', 'I'], dtype=object)

In [46]:
train1.rename({'Yearly Income':'Yearly_Income'}, axis = 1, inplace = True)

In [47]:
test1.rename({'Yearly Income':'Yearly_Income'}, axis = 1, inplace = True)

In [48]:
## Performing statistical test to better understand the above variable

anova_lm(ols('Yearly_Income~GGGrade', data = train1).fit())

,df,sum_sq,mean_sq,F,PR(>F)
GGGrade,6.0,9.216455e+12,1.536076e+12,169.463891,3.841157e-215
Residual,87493.0,7.930651e+14,9.064326e+09,NaN,NaN


In [49]:
train1.rename({'Lend Amount':'Lend_Amount'}, axis = 1, inplace = True)

In [50]:
test1.rename({'Lend Amount':'Lend_Amount'}, axis = 1, inplace = True)

In [51]:
anova_lm(ols('Lend_Amount~GGGrade', data = train1).fit())

,df,sum_sq,mean_sq,F,PR(>F)
GGGrade,6.0,4.876201e+11,8.127001e+10,400.785757,0.0
Residual,87493.0,1.774154e+13,2.027767e+08,NaN,NaN


In [52]:
## We see GGGrade is pretty much dependant on the income of the applicant.

train1.groupby('GGGrade')['Yearly_Income'].median()

GGGrade
I      132000.00
II     114400.00
III    114400.00
IV     111385.12
V      114400.00
VI     114400.00
VII    114400.00
Name: Yearly_Income, dtype: float64

In [53]:
train1.groupby('GGGrade')['Asst_Reg'].median()

GGGrade
I      4331512.0
II     4096504.5
III    4139351.0
IV     3851867.0
V      4217853.5
VI     3738615.5
VII    3178721.0
Name: Asst_Reg, dtype: float64

In [54]:
anova_lm(ols('Asst_Reg~GGGrade', data = train1).fit())

,df,sum_sq,mean_sq,F,PR(>F)
GGGrade,6.0,1.026762e+15,1.711270e+14,32.730786,1.240055e-39
Residual,87493.0,4.574413e+17,5.228319e+12,NaN,NaN


In [55]:
## Although Grading is done basis applicant's income and assets, there isn't any definite order available in the data.
## Thus we might like to encode it as it is in the reverse order to pick up for the model.

def gg(x):
    if x=='I' :
        return(7)
    elif x=='II' :
        return(6)
    elif x=='III' :
        return(5)
    elif x=='IV' :
        return(4)
    elif x=='V' :
        return(3)
    elif x=='VI' :
        return(2)
    else :
        return(1)
    
train1.GGGrade = train1.GGGrade.map(gg)    

In [56]:
test1.GGGrade = test1.GGGrade.map(gg)

In [58]:
train1.Experience.nunique()

11

In [59]:
train1.Experience.unique()

array(['>10yrs', '7yrs', '<1yr', '2yrs', '1yrs', '4yrs', '9yrs', '6yrs',
       '5yrs', '8yrs', '3yrs'], dtype=object)

In [62]:
test1.Experience.unique()

array(['2yrs', '>10yrs', '4yrs', '9yrs', '1yrs', '<1yr', '3yrs', '5yrs',
       '8yrs', '7yrs', '6yrs'], dtype=object)

In [65]:
## We may bin Experience into 4 buckets as below.

def ex(x) :
    if (x == '<1yr') & (x == '1yrs') :
        return('Fresher')
    elif x in ['2yrs','4yrs','3yrs','5yrs'] :
        return('Amateur')
    elif x in ['7yrs','9yrs','6yrs','8yrs'] :
        return('Pro')
    else :
        return('TL')

In [66]:
train1.Experience = train1.Experience.map(ex)
test1.Experience = test1.Experience.map(ex)

In [69]:
## We may thus evaluate points basis experience bins

def exp(x) :
    if x == 'Fresher' :
        return(1)
    elif x == 'Amateur' :
        return(2)
    elif x == 'Pro' :
        return(3)
    else :
        return(4)
    
train1.Experience = train1.Experience.map(exp)
test1.Experience = test1.Experience.map(exp)    

In [72]:
train1.Validation.unique()

array(['Vfied', 'Source Verified', 'Not Vfied'], dtype=object)

In [73]:
test1.Validation.unique()

array(['Source Verified', 'Vfied', 'Not Vfied'], dtype=object)

In [74]:
## We may dummy encode Validation

In [81]:
train1['Home Status'].nunique()

5

In [83]:
train1['Home Status'].unique()

array(['MOR??AGE', 'REN?', 'OWN', 'OTHER', 'NONE'], dtype=object)

In [85]:
train1.groupby('Home Status')['Yearly_Income'].median()

Home Status
MOR??AGE    124960.0
NONE        101200.0
OTHER        90640.0
OWN         109120.0
REN?        105600.0
Name: Yearly_Income, dtype: float64

In [86]:
train1.groupby('Home Status')['Asst_Reg'].median()

Home Status
MOR??AGE    4106617.0
NONE         258202.0
OTHER        215674.0
OWN         4342633.0
REN?        4133238.5
Name: Asst_Reg, dtype: float64

In [87]:
## Thus it may seem fit to bin Home Status into 3 cats

def hs(x) :
    if x in ['MOR??AGE','OWN'] :
        return(3)
    elif x in ['REN?'] :
        return(2)
    else :
        return(1)
    
train1['Home Status'] = train1['Home Status'].map(hs)
test1['Home Status'] = test1['Home Status'].map(hs)

In [91]:
## We may remove the redundant variable Postal code, as demography has been already captured by the State field

In [92]:
train2 = train1.drop('Postal Code', axis = 1)
test2 = test1.drop('Postal Code', axis = 1)

In [98]:
train2['Deprecatory Records'].unique()        ## More the number of such records , less is the chance of loan approval.

array([ 0,  1,  2,  5,  4,  3,  7,  6,  9,  8, 10, 12, 86, 11, 13, 16, 22,
       15], dtype=int64)

In [99]:
## We may thus bin them into 3 categories - 0 (4 pointer), 1-5 (3 pointer), 6-10 (2 pointer),10-30(1 pointer), rest 0 pointers.

In [100]:
def dr(x) :
    if x == 0 :
        return(4)
    elif x in np.arange(1,6) :
        return(3)
    elif x in np.arange(6,11) :
        return(2)
    elif x in np.arange(11,31) :
        return(1)
    else :
        return(0)

In [101]:
train2['Deprecatory Records'] = train2['Deprecatory Records'].map(dr)
test2['Deprecatory Records'] = test2['Deprecatory Records'].map(dr)

In [105]:
train2['Sub GGGrade'].nunique()

35

In [106]:
test2['Sub GGGrade'].nunique()

35

In [107]:
test2['Sub GGGrade'].dtype

dtype('int64')

In [110]:
train2['File Status'].unique()

array(['fully paid', 'whole'], dtype=object)

In [111]:
test2['File Status'].unique()    

array(['fully paid', 'whole'], dtype=object)

In [112]:
pd.crosstab(train2['File Status'],train2['Loan/No Loan'])

Loan/No Loan,0,1
File Status,,
fully paid,30979,10221
whole,40009,6291


In [113]:
stats.chi2_contingency(pd.crosstab(train2['File Status'],train2['Loan/No Loan']))

(1792.1325746621703,
 0.0,
 1,
 array([[33425.20685714,  7774.79314286],
        [37562.79314286,  8737.20685714]]))

In [114]:
## There appears some relation of file status with loan approval, can thus safely encode the same with common sensical pointers.

def fs(x) :
    if x == 'fully paid' :
        return(2)
    else :
        return(1)

In [115]:
train2['File Status'] = train2['File Status'].map(fs)
test2['File Status'] = test2['File Status'].map(fs)

In [120]:
train2.groupby('State')['Loan/No Loan'].count()  ## we have to dummy encode it

State
AK              221
AL             1140
AR              650
AZ             2058
CO             1882
CT             1325
California    12625
DC              235
DE              238
Florida        5813
GA             2924
HI              465
IL             3502
IN             1433
KS              819
KY              821
LA             1019
MA             2025
ME               49
MI             2253
MN             1673
MO             1410
MS              410
MT              259
Maryland       2073
NC             2520
ND               50
NE              134
NH              427
NM              497
NV             1172
NewJersey      3253
Newyork        7210
OK              766
OR             1050
Ohio           2969
PA             3169
RI              372
SC             1005
SD              181
TN             1361
TX             7130
UT              639
VA             2545
VT              178
WA             1860
WI             1116
WV              388
WY              186
Name: Loan/No 

In [123]:
train2.Duration = train2.Duration.str[0]
train2.Duration = train2.Duration.astype(int)

In [124]:
test2.Duration = test2.Duration.str[0]
test2.Duration = test2.Duration.astype(int)

In [129]:
train2.Reason.nunique()

13

In [130]:
stats.chi2_contingency(pd.crosstab(train2.Reason,train2['Loan/No Loan']))   ## Seems to have statistical significance for loan approval

(213.9640463739603,
 4.229149735817299e-39,
 12,
 array([[5.76016914e+02, 1.33983086e+02],
        [1.71198717e+04, 3.98212827e+03],
        [4.26406662e+04, 9.91833381e+03],
        [3.96396992e+03, 9.22030080e+02],
        [2.52311634e+02, 5.86883657e+01],
        [1.22667264e+03, 2.85327360e+02],
        [6.66070263e+02, 1.54929737e+02],
        [4.08079589e+02, 9.49204114e+01],
        [3.06181385e+03, 7.12186149e+02],
        [3.08290743e+01, 7.17092571e+00],
        [6.09279863e+02, 1.41720137e+02],
        [3.43987566e+02, 8.00124343e+01],
        [8.84307657e+01, 2.05692343e+01]]))

In [131]:
##We may need to dummy encode it.

In [132]:
train2['Claim Type'].unique()  ## might dummy encode it

array(['I', 'J'], dtype=object)

In [137]:
train2.head()

,ID,Asst_Reg,GGGrade,Experience,Validation,Yearly_Income,Home Status,Unpaid 2 years,Already Defaulted,Debt to Income,Lend_Amount,Deprecatory Records,Interest Charged,Usage Rate,Inquiries,Present Balance,Gross Collection,Sub GGGrade,File Status,State,Account Open,Total Unpaid CL,Duration,Unpaid Amount,Reason,Claim Type,Due Fee,Loan/No Loan
0,95074,421802,6,4,Vfied,633600.00,3,0,0,12.5496,42023.25,4,15.39,88.924,3,607161.90,0.0,19,2,California,17,58598.0,3,31216.05,debt consolidation,I,0.0,0
1,134131,3964312,4,3,Source Verified,85483.20,2,0,0,9.0048,38133.00,4,9.94,102.856,0,269234.06,0.0,30,1,NC,15,18924.0,5,11660.49,debt consolidation,I,0.0,0
2,92602,4247560,5,4,Source Verified,79200.00,2,0,0,24.9144,17100.00,4,22.35,60.372,0,22476.53,0.0,25,1,Florida,7,15604.0,5,5637.87,major purchase,I,0.0,0
3,22591,197179,5,4,Vfied,61600.00,2,0,0,16.5816,5130.00,4,10.36,116.272,1,15242.09,0.0,22,2,NewJersey,9,22410.0,3,15607.17,major purchase,I,0.0,1
4,125060,4646684,3,2,Source Verified,68053.92,2,0,0,40.0512,19665.00,4,13.68,127.280,1,65433.94,0.0,32,2,LA,10,36022.0,5,27472.86,debt consolidation,I,0.0,0


In [138]:
test2.head()

,ID,Asst_Reg,GGGrade,Experience,Validation,Yearly_Income,Home Status,Unpaid 2 years,Already Defaulted,Debt to Income,Lend_Amount,Deprecatory Records,Interest Charged,Usage Rate,Inquiries,Present Balance,Gross Collection,Sub GGGrade,File Status,State,Account Open,Total Unpaid CL,Duration,Unpaid Amount,Reason,Claim Type,Due Fee
0,198980,1773284,6,2,Source Verified,140800.00,3,0,0,16.9176,17955.00,4,8.88,78.432,1,59769.30,0.0,17,2,California,23,33532.0,3,15755.94,major purchase,I,0.0
1,167688,5298273,6,4,Vfied,146080.00,3,0,0,28.6944,59850.00,4,10.10,76.368,1,532962.13,0.0,19,1,Florida,17,149234.0,5,68237.55,debt consolidation,I,0.0
2,198768,189781,2,4,Vfied,202222.24,3,0,0,51.1560,59850.00,4,19.30,157.552,2,288816.48,0.0,37,1,NM,15,70218.0,5,66272.76,debt consolidation,I,0.0
3,192564,1530906,6,4,Vfied,105600.00,3,1,0,27.0144,19323.00,4,8.82,32.852,2,424664.30,0.0,19,2,DE,18,79182.0,3,15595.20,credit card,I,0.0
4,151199,4410316,5,2,Not Vfied,151360.00,3,0,0,10.3824,25350.75,3,13.52,108.532,1,251704.07,0.0,25,2,Florida,7,13612.0,3,8850.96,debt consolidation,I,0.0


In [139]:
## Separating Numerical , Un-encoded Categorical and Encoded Cat Columns

In [140]:
cat_train = train2.drop('')select_dtypes(include = 'object')
cat_test = test2.select_dtypes(include = 'object')

In [142]:
enc_train = train2.iloc[:,[2,3,6,11,17,18]]
enc_test = test2.iloc[:,[2,3,6,11,17,18]]

In [154]:
num_train = train2.iloc[:,[1,5,7,8,9,10,12,13,14,15,16,20,21,22,23,26]]
num_test = test2.iloc[:,[1,5,7,8,9,10,12,13,14,15,16,20,21,22,23,26]]             ## Invariably dropping ID

#### Dummy encoding required cat vars

In [150]:
dum_train = pd.get_dummies(cat_train, drop_first = True)
dum_test = pd.get_dummies(cat_test, drop_first = True)

#### Power Transforming and then Scaling requisite Num vars

In [155]:
num_train.head()

,Asst_Reg,Yearly_Income,Unpaid 2 years,Already Defaulted,Debt to Income,Lend_Amount,Interest Charged,Usage Rate,Inquiries,Present Balance,Gross Collection,Account Open,Total Unpaid CL,Duration,Unpaid Amount,Due Fee
0,421802,633600.00,0,0,12.5496,42023.25,15.39,88.924,3,607161.90,0.0,17,58598.0,3,31216.05,0.0
1,3964312,85483.20,0,0,9.0048,38133.00,9.94,102.856,0,269234.06,0.0,15,18924.0,5,11660.49,0.0
2,4247560,79200.00,0,0,24.9144,17100.00,22.35,60.372,0,22476.53,0.0,7,15604.0,5,5637.87,0.0
3,197179,61600.00,0,0,16.5816,5130.00,10.36,116.272,1,15242.09,0.0,9,22410.0,3,15607.17,0.0
4,4646684,68053.92,0,0,40.0512,19665.00,13.68,127.280,1,65433.94,0.0,10,36022.0,5,27472.86,0.0


In [156]:
pt = PowerTransformer()

x1 = pt.fit_transform(num_train)

pt_train = pd.DataFrame(x1, columns = num_train.columns)
pt_train.head()

,Asst_Reg,Yearly_Income,Unpaid 2 years,Already Defaulted,Debt to Income,Lend_Amount,Interest Charged,Usage Rate,Inquiries,Present Balance,Gross Collection,Account Open,Total Unpaid CL,Duration,Unpaid Amount,Due Fee
0,-1.640427,3.043477,-0.504101,-0.073094,-1.399277,1.100189,-0.194203,-0.143064,1.610955,1.361789,-0.140126,0.899614,0.506673,-0.675908,0.435428,-0.106857
1,0.181031,-0.609435,-0.504101,-0.073094,-1.742911,0.897028,-1.369854,0.197194,-0.853407,0.560542,-0.140126,0.585931,-0.919903,1.479490,-0.618930,-0.106857
2,0.293040,-0.773392,-0.504101,-0.073094,-0.370685,-0.496647,1.172312,-0.854247,-0.853407,-1.358148,-0.140126,-1.260819,-1.152693,1.479490,-1.265352,-0.106857
3,-1.844974,-1.327621,-0.504101,-0.073094,-1.040598,-1.874379,-1.274723,0.521493,0.916663,-1.598534,-0.140126,-0.666160,-0.713360,-0.675908,-0.329764,-0.106857
4,0.446912,-1.105192,-0.504101,-0.073094,0.696448,-0.286458,-0.550738,0.785418,0.916663,-0.619417,-0.140126,-0.412206,-0.120812,1.479490,0.285701,-0.106857


In [157]:
x2 = pt.transform(num_test)
pt_test = pd.DataFrame(x2, columns = num_test.columns)
pt_test.head()

,Asst_Reg,Yearly_Income,Unpaid 2 years,Already Defaulted,Debt to Income,Lend_Amount,Interest Charged,Usage Rate,Inquiries,Present Balance,Gross Collection,Account Open,Total Unpaid CL,Duration,Unpaid Amount,Due Fee
0,-0.800811,0.413899,-0.504101,-0.073094,-1.011920,-0.424677,-1.614134,-0.402002,0.916663,-0.686729,-0.140126,1.665623,-0.211459,-0.675908,-0.320045,-0.106857
1,0.689366,0.486177,-0.504101,-0.073094,-0.089461,1.914306,-1.333508,-0.453245,0.916663,1.226668,-0.140126,0.899614,1.771570,1.479490,1.447781,-0.106857
2,-1.852725,1.106359,-0.504101,-0.073094,1.402051,1.914306,0.588168,1.502581,1.398267,0.625914,-0.140126,0.585931,0.745271,1.479490,1.406813,-0.106857
3,-0.928986,-0.165962,1.959257,-0.073094,-0.213005,-0.313529,-1.628151,-1.565368,1.398267,0.997440,-0.140126,1.043595,0.905348,-0.675908,-0.330550,-0.106857
4,0.356320,0.555483,-0.504101,-0.073094,-1.605525,0.128749,-0.584615,0.334773,0.916663,0.498500,-0.140126,-1.260819,-1.315684,-0.675908,-0.876152,-0.106857


In [158]:
sc = StandardScaler()

y1 = sc.fit_transform(pt_train)

scaled_train = pd.DataFrame(y1, columns = pt_train.columns)
scaled_train.head()

,Asst_Reg,Yearly_Income,Unpaid 2 years,Already Defaulted,Debt to Income,Lend_Amount,Interest Charged,Usage Rate,Inquiries,Present Balance,Gross Collection,Account Open,Total Unpaid CL,Duration,Unpaid Amount,Due Fee
0,-1.640427,3.043477,-0.504101,-0.073094,-1.399277,1.100189,-0.194203,-0.143064,1.610955,1.361789,-0.140126,0.899614,0.506673,-0.675908,0.435428,-0.106857
1,0.181031,-0.609435,-0.504101,-0.073094,-1.742911,0.897028,-1.369854,0.197194,-0.853407,0.560542,-0.140126,0.585931,-0.919903,1.479490,-0.618930,-0.106857
2,0.293040,-0.773392,-0.504101,-0.073094,-0.370685,-0.496647,1.172312,-0.854247,-0.853407,-1.358148,-0.140126,-1.260819,-1.152693,1.479490,-1.265352,-0.106857
3,-1.844974,-1.327621,-0.504101,-0.073094,-1.040598,-1.874379,-1.274723,0.521493,0.916663,-1.598534,-0.140126,-0.666160,-0.713360,-0.675908,-0.329764,-0.106857
4,0.446912,-1.105192,-0.504101,-0.073094,0.696448,-0.286458,-0.550738,0.785418,0.916663,-0.619417,-0.140126,-0.412206,-0.120812,1.479490,0.285701,-0.106857


In [159]:
y2 = sc.transform(pt_test)

scaled_test = pd.DataFrame(y2, columns = pt_test.columns)
scaled_test.head()

,Asst_Reg,Yearly_Income,Unpaid 2 years,Already Defaulted,Debt to Income,Lend_Amount,Interest Charged,Usage Rate,Inquiries,Present Balance,Gross Collection,Account Open,Total Unpaid CL,Duration,Unpaid Amount,Due Fee
0,-0.800811,0.413899,-0.504101,-0.073094,-1.011920,-0.424677,-1.614134,-0.402002,0.916663,-0.686729,-0.140126,1.665623,-0.211459,-0.675908,-0.320045,-0.106857
1,0.689366,0.486177,-0.504101,-0.073094,-0.089461,1.914306,-1.333508,-0.453245,0.916663,1.226668,-0.140126,0.899614,1.771570,1.479490,1.447781,-0.106857
2,-1.852725,1.106359,-0.504101,-0.073094,1.402051,1.914306,0.588168,1.502581,1.398267,0.625914,-0.140126,0.585931,0.745271,1.479490,1.406813,-0.106857
3,-0.928986,-0.165962,1.959257,-0.073094,-0.213005,-0.313529,-1.628151,-1.565368,1.398267,0.997440,-0.140126,1.043595,0.905348,-0.675908,-0.330550,-0.106857
4,0.356320,0.555483,-0.504101,-0.073094,-1.605525,0.128749,-0.584615,0.334773,0.916663,0.498500,-0.140126,-1.260819,-1.315684,-0.675908,-0.876152,-0.106857


#### Preparing Scaled Data

In [160]:
dum_train = dum_train.reset_index(drop = True)
enc_train = enc_train.reset_index(drop = True)
scaled_train = scaled_train.reset_index(drop = True)


dum_test = dum_test.reset_index(drop = True)
enc_test = enc_test.reset_index(drop = True)
scaled_test = scaled_test.reset_index(drop = True)

In [161]:
conctr1 = pd.concat([dum_train,enc_train], axis = 1)
concts1 = pd.concat([dum_test,enc_test], axis = 1)

In [162]:
conctr1 = conctr1.reset_index(drop = True)
concts1 = concts1.reset_index(drop = True)

In [163]:
final_scaled_train = pd.concat([scaled_train,conctr1], axis = 1)
final_scaled_test = pd.concat([scaled_test,concts1], axis = 1)

In [164]:
final_scaled_train.head()

,Asst_Reg,Yearly_Income,Unpaid 2 years,Already Defaulted,Debt to Income,Lend_Amount,Interest Charged,Usage Rate,Inquiries,Present Balance,Gross Collection,Account Open,Total Unpaid CL,Duration,Unpaid Amount,Due Fee,Validation_Source Verified,Validation_Vfied,State_AL,State_AR,State_AZ,State_CO,State_CT,State_California,State_DC,State_DE,State_Florida,State_GA,State_HI,State_IL,State_IN,State_KS,State_KY,State_LA,State_MA,State_ME,State_MI,State_MN,State_MO,State_MS,State_MT,State_Maryland,State_NC,State_ND,State_NE,State_NH,State_NM,State_NV,State_NewJersey,State_Newyork,State_OK,State_OR,State_Ohio,State_PA,State_RI,State_SC,State_SD,State_TN,State_TX,State_UT,State_VA,State_VT,State_WA,State_WI,State_WV,State_WY,Reason_credit card,Reason_debt consolidation,Reason_home improvement,Reason_house,Reason_major purchase,Reason_medical,Reason_moving,Reason_other,Reason_renewable energy,Reason_small business,Reason_vacation,Reason_wedding,Claim Type_J,GGGrade,Experience,Home Status,Deprecatory Records,Sub GGGrade,File Status
0,-1.640427,3.043477,-0.504101,-0.073094,-1.399277,1.100189,-0.194203,-0.143064,1.610955,1.361789,-0.140126,0.899614,0.506673,-0.675908,0.435428,-0.106857,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,6,4,3,4,19,2
1,0.181031,-0.609435,-0.504101,-0.073094,-1.742911,0.897028,-1.369854,0.197194,-0.853407,0.560542,-0.140126,0.585931,-0.919903,1.479490,-0.618930,-0.106857,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,4,3,2,4,30,1
2,0.293040,-0.773392,-0.504101,-0.073094,-0.370685,-0.496647,1.172312,-0.854247,-0.853407,-1.358148,-0.140126,-1.260819,-1.152693,1.479490,-1.265352,-0.106857,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,5,4,2,4,25,1
3,-1.844974,-1.327621,-0.504101,-0.073094,-1.040598,-1.874379,-1.274723,0.521493,0.916663,-1.598534,-0.140126,-0.666160,-0.713360,-0.675908,-0.329764,-0.106857,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,5,4,2,4,22,2
4,0.446912,-1.105192,-0.504101,-0.073094,0.696448,-0.286458,-0.550738,0.785418,0.916663,-0.619417,-0.140126,-0.412206,-0.120812,1.479490,0.285701,-0.106857,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,3,2,2,4,32,2


In [167]:
final_scaled_train.shape

(87500, 85)

In [165]:
final_scaled_test.head()

,Asst_Reg,Yearly_Income,Unpaid 2 years,Already Defaulted,Debt to Income,Lend_Amount,Interest Charged,Usage Rate,Inquiries,Present Balance,Gross Collection,Account Open,Total Unpaid CL,Duration,Unpaid Amount,Due Fee,Validation_Source Verified,Validation_Vfied,State_AL,State_AR,State_AZ,State_CO,State_CT,State_California,State_DC,State_DE,State_Florida,State_GA,State_HI,State_IL,State_IN,State_KS,State_KY,State_LA,State_MA,State_ME,State_MI,State_MN,State_MO,State_MS,State_MT,State_Maryland,State_NC,State_ND,State_NE,State_NH,State_NM,State_NV,State_NewJersey,State_Newyork,State_OK,State_OR,State_Ohio,State_PA,State_RI,State_SC,State_SD,State_TN,State_TX,State_UT,State_VA,State_VT,State_WA,State_WI,State_WV,State_WY,Reason_credit card,Reason_debt consolidation,Reason_home improvement,Reason_house,Reason_major purchase,Reason_medical,Reason_moving,Reason_other,Reason_renewable energy,Reason_small business,Reason_vacation,Reason_wedding,Claim Type_J,GGGrade,Experience,Home Status,Deprecatory Records,Sub GGGrade,File Status
0,-0.800811,0.413899,-0.504101,-0.073094,-1.011920,-0.424677,-1.614134,-0.402002,0.916663,-0.686729,-0.140126,1.665623,-0.211459,-0.675908,-0.320045,-0.106857,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,6,2,3,4,17,2
1,0.689366,0.486177,-0.504101,-0.073094,-0.089461,1.914306,-1.333508,-0.453245,0.916663,1.226668,-0.140126,0.899614,1.771570,1.479490,1.447781,-0.106857,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,6,4,3,4,19,1
2,-1.852725,1.106359,-0.504101,-0.073094,1.402051,1.914306,0.588168,1.502581,1.398267,0.625914,-0.140126,0.585931,0.745271,1.479490,1.406813,-0.106857,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,4,3,4,37,1
3,-0.928986,-0.165962,1.959257,-0.073094,-0.213005,-0.313529,-1.628151,-1.565368,1.398267,0.997440,-0.140126,1.043595,0.905348,-0.675908,-0.330550,-0.106857,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,6,4,3,4,19,2
4,0.356320,0.555483,-0.504101,-0.073094,-1.605525,0.128749,-0.584615,0.334773,0.916663,0.498500,-0.140126,-1.260819,-1.315684,-0.675908,-0.876152,-0.106857,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5,2,3,3,25,2


In [168]:
final_scaled_test.shape

(37500, 85)

#### Preparing Unscaled Data

In [166]:
num_train = num_train.reset_index(drop = True)
num_test = num_test.reset_index(drop = True)

In [169]:
final_train = pd.concat([num_train,conctr1], axis = 1)
final_test = pd.concat([num_test,concts1], axis = 1)

In [170]:
final_train.head()

,Asst_Reg,Yearly_Income,Unpaid 2 years,Already Defaulted,Debt to Income,Lend_Amount,Interest Charged,Usage Rate,Inquiries,Present Balance,Gross Collection,Account Open,Total Unpaid CL,Duration,Unpaid Amount,Due Fee,Validation_Source Verified,Validation_Vfied,State_AL,State_AR,State_AZ,State_CO,State_CT,State_California,State_DC,State_DE,State_Florida,State_GA,State_HI,State_IL,State_IN,State_KS,State_KY,State_LA,State_MA,State_ME,State_MI,State_MN,State_MO,State_MS,State_MT,State_Maryland,State_NC,State_ND,State_NE,State_NH,State_NM,State_NV,State_NewJersey,State_Newyork,State_OK,State_OR,State_Ohio,State_PA,State_RI,State_SC,State_SD,State_TN,State_TX,State_UT,State_VA,State_VT,State_WA,State_WI,State_WV,State_WY,Reason_credit card,Reason_debt consolidation,Reason_home improvement,Reason_house,Reason_major purchase,Reason_medical,Reason_moving,Reason_other,Reason_renewable energy,Reason_small business,Reason_vacation,Reason_wedding,Claim Type_J,GGGrade,Experience,Home Status,Deprecatory Records,Sub GGGrade,File Status
0,421802,633600.00,0,0,12.5496,42023.25,15.39,88.924,3,607161.90,0.0,17,58598.0,3,31216.05,0.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,6,4,3,4,19,2
1,3964312,85483.20,0,0,9.0048,38133.00,9.94,102.856,0,269234.06,0.0,15,18924.0,5,11660.49,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,4,3,2,4,30,1
2,4247560,79200.00,0,0,24.9144,17100.00,22.35,60.372,0,22476.53,0.0,7,15604.0,5,5637.87,0.0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,5,4,2,4,25,1
3,197179,61600.00,0,0,16.5816,5130.00,10.36,116.272,1,15242.09,0.0,9,22410.0,3,15607.17,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,5,4,2,4,22,2
4,4646684,68053.92,0,0,40.0512,19665.00,13.68,127.280,1,65433.94,0.0,10,36022.0,5,27472.86,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,3,2,2,4,32,2


In [172]:
final_train.shape

(87500, 85)

In [171]:
final_test.head()

,Asst_Reg,Yearly_Income,Unpaid 2 years,Already Defaulted,Debt to Income,Lend_Amount,Interest Charged,Usage Rate,Inquiries,Present Balance,Gross Collection,Account Open,Total Unpaid CL,Duration,Unpaid Amount,Due Fee,Validation_Source Verified,Validation_Vfied,State_AL,State_AR,State_AZ,State_CO,State_CT,State_California,State_DC,State_DE,State_Florida,State_GA,State_HI,State_IL,State_IN,State_KS,State_KY,State_LA,State_MA,State_ME,State_MI,State_MN,State_MO,State_MS,State_MT,State_Maryland,State_NC,State_ND,State_NE,State_NH,State_NM,State_NV,State_NewJersey,State_Newyork,State_OK,State_OR,State_Ohio,State_PA,State_RI,State_SC,State_SD,State_TN,State_TX,State_UT,State_VA,State_VT,State_WA,State_WI,State_WV,State_WY,Reason_credit card,Reason_debt consolidation,Reason_home improvement,Reason_house,Reason_major purchase,Reason_medical,Reason_moving,Reason_other,Reason_renewable energy,Reason_small business,Reason_vacation,Reason_wedding,Claim Type_J,GGGrade,Experience,Home Status,Deprecatory Records,Sub GGGrade,File Status
0,1773284,140800.00,0,0,16.9176,17955.00,8.88,78.432,1,59769.30,0.0,23,33532.0,3,15755.94,0.0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,6,2,3,4,17,2
1,5298273,146080.00,0,0,28.6944,59850.00,10.10,76.368,1,532962.13,0.0,17,149234.0,5,68237.55,0.0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,6,4,3,4,19,1
2,189781,202222.24,0,0,51.1560,59850.00,19.30,157.552,2,288816.48,0.0,15,70218.0,5,66272.76,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,4,3,4,37,1
3,1530906,105600.00,1,0,27.0144,19323.00,8.82,32.852,2,424664.30,0.0,18,79182.0,3,15595.20,0.0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,6,4,3,4,19,2
4,4410316,151360.00,0,0,10.3824,25350.75,13.52,108.532,1,251704.07,0.0,7,13612.0,3,8850.96,0.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5,2,3,3,25,2


In [173]:
final_test.shape

(37500, 85)

### Base Model (LG)

In [174]:
y = train2['Loan/No Loan']

In [175]:
lg = LogisticRegression()

lg.fit(final_scaled_train,y)

LogisticRegression()

In [176]:
lg_pred = lg.predict(final_scaled_test)

In [195]:
mod1 = pd.DataFrame({'ID':test.ID,'Loan/No Loan':lg_pred})
mod1.head()

,ID,Loan/No Loan
0,198980,0
1,167688,0
2,198768,1
3,192564,0
4,151199,0


In [196]:
mod1.set_index('ID', inplace = True)

In [197]:
mod1.to_csv('Base_Lg.csv', index_label = False)

### DT

In [200]:
dt = DecisionTreeClassifier()

dt.fit(final_train,y)

DecisionTreeClassifier()

In [201]:
dt_pred = dt.predict(final_test)

In [202]:
mod2 = pd.DataFrame({'ID':test.ID,'Loan/No Loan':dt_pred})
mod2.head()

,ID,Loan/No Loan
0,198980,0
1,167688,1
2,198768,0
3,192564,1
4,151199,1


In [203]:
mod2.set_index('ID', inplace = True)

In [204]:
mod2.to_csv('DT1.csv', index_label = False)

### RF

In [205]:
rf = RandomForestClassifier()

rf.fit(final_train,y)

RandomForestClassifier()

In [206]:
rf_pred = rf.predict(final_test)

In [207]:
mod3 = pd.DataFrame({'ID':test.ID,'Loan/No Loan':rf_pred})
mod3.head()

,ID,Loan/No Loan
0,198980,0
1,167688,0
2,198768,0
3,192564,0
4,151199,0


In [208]:
mod3.set_index('ID', inplace = True)

In [209]:
mod3.to_csv('RF1.csv', index_label = False)

#### It seems DT is doing a better job than both RF and LG

### Hyper-Parameter Tuning DT

In [211]:
(train2['Loan/No Loan'].value_counts()/train2.shape[0])*100     ## Not a very much imbalance to use SMOTE; might use class weight as balanced

0    81.129143
1    18.870857
Name: Loan/No Loan, dtype: float64

In [212]:
import sklearn
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

In [214]:
tuned = [{'criterion':['gini','entropy'],
         'max_depth':np.arange(2,15),
         'min_samples_split':np.arange(2,15),
         'min_samples_leaf':np.arange(2,15),
         'max_features':np.arange(2,6)}]

dt2 = DecisionTreeClassifier(class_weight = 'balanced' , random_state = 10)

grid = GridSearchCV(estimator = dt2, param_grid = tuned, scoring = 'f1', cv = 5, n_jobs = -1 )

grid.fit(final_train,y)

grid.best_params_

{'criterion': 'gini',
 'max_depth': 9,
 'max_features': 5,
 'min_samples_leaf': 14,
 'min_samples_split': 2}

In [215]:
dt_tuned = DecisionTreeClassifier(criterion = 'gini',max_depth = 9,max_features = 5, min_samples_leaf = 14, min_samples_split = 2, class_weight = 'balanced', random_state = 10)

dt_tuned.fit(final_train,y)

DecisionTreeClassifier(class_weight='balanced', max_depth=9, max_features=5,
                       min_samples_leaf=14, random_state=10)

In [216]:
tuned_dt_pred = dt_tuned.predict(final_test)

In [217]:
mod4 = pd.DataFrame({'ID':test.ID,'Loan/No Loan':tuned_dt_pred})
mod4.head()

,ID,Loan/No Loan
0,198980,1
1,167688,0
2,198768,1
3,192564,1
4,151199,0


In [218]:
mod4.set_index('ID', inplace = True)

In [219]:
mod4.to_csv('DT2.csv', index_label = False)

### Feature Importances of tuned DT

In [222]:
imp_ftrs = pd.DataFrame({'Features':final_train.columns,
                        'Importances':dt_tuned.feature_importances_})

imp_ftrs = imp_ftrs.sort_values(by = 'Importances', ascending = False)

imp_ftrs

,Features,Importances
0,Asst_Reg,0.901178
10,Gross Collection,0.020187
79,GGGrade,0.010590
84,File Status,0.008272
7,Usage Rate,0.008242
16,Validation_Source Verified,0.006654
4,Debt to Income,0.005667
5,Lend_Amount,0.004353
14,Unpaid Amount,0.004135
12,Total Unpaid CL,0.003666


### Feature Selection Techniques

In [223]:
sfs_mod = SFS(estimator = dt_tuned , k_features = 'best', forward = True, cv = 2, scoring = 'f1', n_jobs = -1)

sfs_mod.fit(final_train,y)

sfs_mod.k_feature_names_

('Asst_Reg',
 'Yearly_Income',
 'Unpaid 2 years',
 'Already Defaulted',
 'Inquiries',
 'Gross Collection',
 'Duration',
 'Unpaid Amount',
 'Due Fee',
 'State_CT',
 'State_Florida',
 'State_LA',
 'State_MA',
 'State_ME',
 'State_MT',
 'State_ND',
 'State_NE',
 'State_NH',
 'State_WA',
 'Reason_house',
 'Reason_medical',
 'Reason_small  business',
 'Reason_wedding')

#### Building model by taking a combination of features from Feature Importances as well as SFS

In [224]:
feature_train = final_train.loc[:,['Asst_Reg','Yearly_Income','Unpaid 2 years','Already Defaulted','Inquiries','Gross Collection',
                                  'Duration','Unpaid Amount','Due Fee','GGGrade','File Status','Usage Rate','Debt to Income','Lend_Amount',
                                  'Unpaid Amount']]

feature_test = final_test.loc[:,['Asst_Reg','Yearly_Income','Unpaid 2 years','Already Defaulted','Inquiries','Gross Collection',
                                  'Duration','Unpaid Amount','Due Fee','GGGrade','File Status','Usage Rate','Debt to Income','Lend_Amount',
                                  'Unpaid Amount']]

In [225]:
dt_tuned2 = DecisionTreeClassifier(criterion = 'gini',max_depth = 9,max_features = 5, min_samples_leaf = 14, min_samples_split = 2, class_weight = 'balanced', random_state = 10)

dt_tuned2.fit(feature_train,y)

DecisionTreeClassifier(class_weight='balanced', max_depth=9, max_features=5,
                       min_samples_leaf=14, random_state=10)

In [226]:
tuned_feature_pred = dt_tuned2.predict(feature_test)

In [227]:
mod5 = pd.DataFrame({'ID':test.ID,'Loan/No Loan':tuned_feature_pred})
mod5.head()

,ID,Loan/No Loan
0,198980,1
1,167688,0
2,198768,1
3,192564,1
4,151199,0


In [228]:
mod5.set_index('ID', inplace = True)

In [229]:
mod5.to_csv('DT3.csv', index_label = False)

### Using RFE

In [230]:
rfe = RFE(estimator = dt_tuned , n_features_to_select = 10)

mod_rfe = rfe.fit(final_train,y)

feat_index = pd.Series(data = mod_rfe.ranking_ , index = final_train.columns)

signi_feat = feat_index[feat_index == 1].index

print(signi_feat)

Index(['Asst_Reg', 'Debt to Income', 'Interest Charged', 'Usage Rate',
       'Present Balance', 'Gross Collection', 'Total Unpaid CL', 'Duration',
       'Unpaid Amount', 'Sub GGGrade'],
      dtype='object')


### Taking the most common features of all the above methods used

In [231]:
feature_train2 = final_train.loc[:,['Asst_Reg','Debt to Income','Usage Rate','Gross Collection','Duration','Unpaid Amount']]

feature_test2 = final_test.loc[:,['Asst_Reg','Debt to Income','Usage Rate','Gross Collection','Duration','Unpaid Amount']]



In [232]:
dt_tuned3 = DecisionTreeClassifier(criterion = 'gini',max_depth = 9, min_samples_leaf = 14, min_samples_split = 2, class_weight = 'balanced', random_state = 10)

dt_tuned3.fit(feature_train2,y)

DecisionTreeClassifier(class_weight='balanced', max_depth=9,
                       min_samples_leaf=14, random_state=10)

In [233]:
tuned_feature_pred2 = dt_tuned3.predict(feature_test2)

In [234]:
mod6 = pd.DataFrame({'ID':test.ID,'Loan/No Loan':tuned_feature_pred2})
mod6.head()

,ID,Loan/No Loan
0,198980,1
1,167688,0
2,198768,0
3,192564,1
4,151199,0


In [235]:
mod6.set_index('ID', inplace = True)

In [236]:
mod6.to_csv('DT4.csv', index_label = False)

### Trying a hand with Probabilistic Model

In [237]:
gnb = GaussianNB()

gnb.fit(feature_train2,y)

GaussianNB()

In [238]:
gnb_pred = gnb.predict(feature_test2)

In [239]:
mod7 = pd.DataFrame({'ID':test.ID,'Loan/No Loan':gnb_pred})
mod7.head()

,ID,Loan/No Loan
0,198980,1
1,167688,0
2,198768,1
3,192564,1
4,151199,0


In [240]:
mod7.set_index('ID', inplace = True)

In [241]:
mod7.to_csv('gnb.csv', index_label = False)

#### With the above set of features, it didn't really do a bad job, but there was no improvement either

### Trying XGBoost

In [244]:
xgb = XGBClassifier()

xgb_mod = xgb.fit(feature_train2,y)

[02:04:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [245]:
xgb_pred = xgb_mod.predict(feature_test2)

In [246]:
mod8 = pd.DataFrame({'ID':test.ID,'Loan/No Loan':xgb_pred})
mod8.head()

,ID,Loan/No Loan
0,198980,0
1,167688,0
2,198768,0
3,192564,0
4,151199,0


In [247]:
mod8.set_index('ID', inplace = True)

In [248]:
mod8.to_csv('xgb.csv', index_label = False)

#### xgb isn't working here

### Stacking Classifier

In [251]:
base_learners = [('gnb',GaussianNB()),
                ('dt',DecisionTreeClassifier(criterion = 'gini',max_depth = 9, min_samples_leaf = 14, min_samples_split = 2, class_weight = 'balanced', random_state = 10))]

stack_model = StackingClassifier(estimators = base_learners, final_estimator = DecisionTreeClassifier(criterion = 'gini',max_depth = 9, min_samples_leaf = 14, min_samples_split = 2, class_weight = 'balanced', random_state = 10))

stack_model.fit(feature_train2,y)

StackingClassifier(estimators=[('gnb', GaussianNB()),
                               ('dt',
                                DecisionTreeClassifier(class_weight='balanced',
                                                       max_depth=9,
                                                       min_samples_leaf=14,
                                                       random_state=10))],
                   final_estimator=DecisionTreeClassifier(class_weight='balanced',
                                                          max_depth=9,
                                                          min_samples_leaf=14,
                                                          random_state=10))

In [252]:
stack_pred = stack_model.predict(feature_test2)

In [253]:
mod9 = pd.DataFrame({'ID':test.ID,'Loan/No Loan':stack_pred})
mod9.head()

,ID,Loan/No Loan
0,198980,1
1,167688,0
2,198768,1
3,192564,1
4,151199,0


In [254]:
mod9.set_index('ID', inplace = True)

In [255]:
mod9.to_csv('stack.csv', index_label = False)

#### Adding Lend Amount

In [256]:
feature_train3 = final_train.loc[:,['Asst_Reg','Debt to Income','Usage Rate','Gross Collection','Duration','Unpaid Amount','Lend_Amount']]

feature_test3 = final_test.loc[:,['Asst_Reg','Debt to Income','Usage Rate','Gross Collection','Duration','Unpaid Amount','Lend_Amount']]



In [257]:
dt_tuned4 = DecisionTreeClassifier(criterion = 'gini',max_depth = 9, min_samples_leaf = 14, min_samples_split = 2, class_weight = 'balanced', random_state = 10)

dt_tuned4.fit(feature_train3,y)

DecisionTreeClassifier(class_weight='balanced', max_depth=9,
                       min_samples_leaf=14, random_state=10)

In [259]:
tuned_feature_pred3 = dt_tuned4.predict(feature_test3)

In [260]:
mod10 = pd.DataFrame({'ID':test.ID,'Loan/No Loan':tuned_feature_pred3})
mod10.head()

,ID,Loan/No Loan
0,198980,1
1,167688,0
2,198768,0
3,192564,1
4,151199,0


In [261]:
mod10.set_index('ID', inplace = True)

In [262]:
mod10.to_csv('DT5.csv', index_label = False)

#### There isn't any improvement either

### Trying KNN

In [263]:
knn = KNeighborsClassifier()

knn.fit(feature_train2,y)

KNeighborsClassifier()

In [264]:
pred_knn = knn.predict(feature_test2)

In [265]:
mod11 = pd.DataFrame({'ID':test.ID,'Loan/No Loan':pred_knn})
mod11.head()

,ID,Loan/No Loan
0,198980,0
1,167688,0
2,198768,1
3,192564,0
4,151199,0


In [266]:
mod11.set_index('ID', inplace = True)

In [267]:
mod11.to_csv('knn.csv', index_label = False)

### From above iterations  we get to conclude that Random Forest, KNN and xgboost aren't doing good

### Only tuned DT and Gaussian Naive bayes performed relatively better

### Trying SVC

In [269]:
svc = sklearn.svm.SVC(class_weight = 'balanced', random_state = 10)

svc.fit(feature_train2,y)

SVC(class_weight='balanced', random_state=10)

In [270]:
pred_svc = svc.predict(feature_test2)

In [271]:
mod12 = pd.DataFrame({'ID':test.ID,'Loan/No Loan':pred_svc})
mod12.head()

,ID,Loan/No Loan
0,198980,1
1,167688,0
2,198768,1
3,192564,1
4,151199,0


In [272]:
mod12.set_index('ID', inplace = True)

In [273]:
mod12.to_csv('svc.csv', index_label = False)

### Tuned DT by far gave the best result followed by Gnb and SVC
### All the 3 can be stacked and checked

In [275]:
base_learners2 = [('gnb',GaussianNB()),
                ('svc',sklearn.svm.SVC(class_weight = 'balanced', random_state = 10))]

stack_model2 = StackingClassifier(estimators = base_learners2, final_estimator = DecisionTreeClassifier(criterion = 'gini',max_depth = 9, min_samples_leaf = 14, min_samples_split = 2, class_weight = 'balanced', random_state = 10))

stack_model2.fit(feature_train2,y)

StackingClassifier(estimators=[('gnb', GaussianNB()),
                               ('svc',
                                SVC(class_weight='balanced', random_state=10))],
                   final_estimator=DecisionTreeClassifier(class_weight='balanced',
                                                          max_depth=9,
                                                          min_samples_leaf=14,
                                                          random_state=10))

In [276]:
stack_pred2 = stack_model2.predict(feature_test2)

In [277]:
mod13 = pd.DataFrame({'ID':test.ID,'Loan/No Loan':stack_pred2})
mod13.head()

,ID,Loan/No Loan
0,198980,1
1,167688,0
2,198768,1
3,192564,1
4,151199,0


In [278]:
mod13.set_index('ID', inplace = True)

In [279]:
mod13.to_csv('stack2.csv', index_label = False)

In [281]:
base_learners3 = [('gnb',GaussianNB()),
                ('svc',sklearn.svm.SVC(class_weight = 'balanced', random_state = 10)),
                 ('dt', DecisionTreeClassifier(criterion = 'gini',max_depth = 9, min_samples_leaf = 14, min_samples_split = 2, class_weight = 'balanced', random_state = 10))]

stack_model3 = StackingClassifier(estimators = base_learners3, final_estimator = DecisionTreeClassifier(criterion = 'gini',max_depth = 9, min_samples_leaf = 14, min_samples_split = 2, class_weight = 'balanced', random_state = 10))

stack_model3.fit(feature_train2,y)

StackingClassifier(estimators=[('gnb', GaussianNB()),
                               ('svc',
                                SVC(class_weight='balanced', random_state=10)),
                               ('dt',
                                DecisionTreeClassifier(class_weight='balanced',
                                                       max_depth=9,
                                                       min_samples_leaf=14,
                                                       random_state=10))],
                   final_estimator=DecisionTreeClassifier(class_weight='balanced',
                                                          max_depth=9,
                                                          min_samples_leaf=14,
                                                          random_state=10))

In [282]:
stack_pred3 = stack_model3.predict(feature_test2)

In [283]:
mod14 = pd.DataFrame({'ID':test.ID,'Loan/No Loan':stack_pred3})
mod14.head()

,ID,Loan/No Loan
0,198980,1
1,167688,0
2,198768,1
3,192564,1
4,151199,0


In [284]:
mod14.set_index('ID', inplace = True)

In [285]:
mod14.to_csv('stack3.csv', index_label = False)

### No further improvement seen after several iterations.
### Model 9 - 1st attempt on Stacking gave best F1 score of 0.5647553 across all other attempted models